Это переделанная тетрадка, потому что в оригинальной ячейки запускались в совершенно не очевидном порядке. Плюс были те, которые не надо запускать вообще. Если надо, мы можем ее скинуть, но там страшно

In [54]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [33]:
df = pd.read_csv("clever_features.csv").drop(["Unnamed: 0", "ID"], axis=1)
df.head()

,Answer,Question,sentence,text_len,fancy_quotes,regular_quotes,count_question_marks,count_excl_marks,count_dots,count_commas,...,ends_question_mark,good_start_words,first_word,second_word,first_word_len,second_word_len,uppercase_letters,has_bad_spaces,looks_bad,len_class_1
0,0,Как зовут лодочника на реке Стикс в древнегреч...,как зовут лодочника на реке стикс в древнегреч...,10,0,0,1,0,0,0,...,1,1,как,зовут,3,5,2,0,0,0.130242
1,1,Как в химии обозначается свинец?,как в химии обозначается свинец,6,0,0,1,0,0,0,...,1,1,как,в,3,1,1,0,0,0.091091
2,0,Какой химический элемент преобладает в составе...,какой химический элемент преобладает в составе...,9,0,0,1,0,0,0,...,1,1,какой,химический,5,10,1,0,0,0.136674
3,0,Кто из перечисленных был пажом во времена Екат...,кто из перечисленных был пажом во времена екат...,10,0,0,1,0,0,0,...,1,1,кто,из,3,2,4,0,0,0.130242
4,0,Когда началась 2 мировая война?,когда началась 2 мировая война,6,0,0,1,0,0,0,...,1,1,когда,началась,5,8,1,0,0,0.091091


Ниже список функций, которые использовались для генерации дополнительных фичей:

In [59]:
import nltk

def text_len(text):
    return len(nltk.word_tokenize(text.lower()))

def fancy_quotes(text):
    if "«" in text and "»" in text:
        return 1
    else:
        return 0

def regular_quotes(text):
    if '"' in text:
        return 1
    else:
        return 0

def count_question_marks(text):
    return Counter(text)["?"]

def count_excl_marks(text):
    return Counter(text)["!"]

def count_dots(text):
    return Counter(text)["."]

def count_commas(text):
    return Counter(text)[","]

def count_digits(text):
    ans = 0
    for i in range(10):
        ans += Counter(text)[i]
    return ans

def ends_question_mark(text):
    return int(text[-1] == "?")

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
start_words = ["как", "что", "сколько", "какое", "какой", "какая", "кто", "когда", "откуда", "куда", "где"]

def first_word(text):
    tokens = nltk.word_tokenize(text.lower())
    return tokens[0]

def second_word(text):
    tokens = nltk.word_tokenize(text.lower())
    if len(tokens) >= 2:
        return tokens[1]
    else:
        return "none"
    
def good_start_words(text):
    w1 = first_word(text)
    w2 = second_word(text)
    if w2 == "none":
        return 0

    if morph.parse(w1)[0].normal_form in start_words or morph.parse(w2)[0].normal_form in start_words:
        return 1
    else:
        return 0

def first_word_len(text):
    return len(first_word(text))

def second_word_len(text):
    w = second_word(text)
    if w != "none":
        return len(w)
    else:
        return 0

def uppercase_letters(text):
    return sum(1 for c in text if c.isupper())

def clever_vk(text):
    if "клевер" in text.lower() or "вконтакте" in text.lower() or "ВК" in text:
        return 1
    else:
        return 0

In [60]:
import re

def has_bad_spaces(text):
    pattern = r"\s[,.!?;]|[,.!?;][^\s]"
    if re.findall(pattern, text):
        return 1
    else:
        return 0

def looks_bad(text):
    if (
        (text_len(text) >= 25) or
        (regular_quotes(text) > 0) or
        (count_question_marks(text) > 2) or
        (count_excl_marks(text) > 1) or
        (count_dots(text) > 3) or
        (not ends_question_mark(text)) or
        (uppercase_letters(text) > 15) or
        (has_bad_spaces(text)) or
        (clever_vk(text))
    ):
        return 1
    else:
        return 0

def looks_good(text):
    if (
        (6 < text_len(text) < 14) and
        (regular_quotes(text) == 0) and
        (ends_question_mark(text)) and
        (not has_bad_spaces(text))
    ):
        return 1
    return 0

In [105]:
from collections import defaultdict

len_to_ans = defaultdict(list)
for idx, row in df.iterrows():
    len_to_ans[row["text_len"]].append(row["Answer"])

len_to_ans = {k: sum(v) / len(v) for k, v in len_to_ans.items()}
def len_class_1(text):
    len_ = text_len(text)
    return len_to_ans.get(len_, 0)

#df["len_class_1"] = df["text_len"].apply(lambda x: len_to_ans[x])

# это признак "сколько (доля) в исходном датасете единиц среди текстов такой длины"

In [109]:
def ok_upper(text):
  if uppercase_letters(text) < 7:
    return 1
  return 0

def ok_upper_text(text):
  if re.match(r"^[А-ЯЁ].*\?$", text) and not re.match(r'[А-ЯЁ]{4,}', text):
    return 1
  return 0

In [110]:
# мы не руками это сделали, а в vim, если что

new_features = {
    "text_len": text_len,
    "fancy_quotes": fancy_quotes,
    "regular_quotes": regular_quotes,
    "count_question_marks": count_question_marks,
    "count_excl_marks": count_excl_marks,
    "count_dots": count_dots,
    "count_commas": count_commas,
    "count_digits": count_digits,
    "ends_question_mark": ends_question_mark,
    "good_start_words": good_start_words,
    "first_word": first_word,
    "second_word": second_word,
    "first_word_len": first_word_len,
    "second_word_len": second_word_len,
    "uppercase_letters": uppercase_letters,
    "has_bad_spaces": has_bad_spaces,
    "looks_bad": looks_bad,
    "len_class_1": len_class_1,
    "clever_vk": clever_vk,
    "looks_good": looks_good,
    "ok_upper": ok_upper,
    "ok_upper_text": ok_upper_text
}

In [112]:
for name, func in tqdm(new_features.items()):
    df[name] = df["Question"].apply(func)

  0%|          | 0/22 [00:00<?, ?it/s]

# Добавление эмбеддингов

In [131]:
import pickle

with open("q_emb.pickle", "rb") as f:
    q_emb = pickle.load(f)

In [132]:
q_emb = np.array([emb.detach().numpy() for emb in q_emb])  # там тензоры

In [133]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
q_emb_100 = pca.fit_transform(q_emb)

In [134]:
df_with_emb = pd.concat([df, pd.DataFrame(q_emb_100)], axis=1)

# CatBoost

In [117]:
df.columns[1:]

Index(['Question', 'sentence', 'text_len', 'fancy_quotes', 'regular_quotes',
       'count_question_marks', 'count_excl_marks', 'count_dots',
       'count_commas', 'count_digits', 'ends_question_mark',
       'good_start_words', 'first_word', 'second_word', 'first_word_len',
       'second_word_len', 'uppercase_letters', 'has_bad_spaces', 'looks_bad',
       'len_class_1', 'clever_vk', 'looks_good', 'ok_upper', 'ok_upper_text'],
      dtype='object')

In [118]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[df.columns[1:]], df["Answer"], train_size=.8, random_state=1337)

In [119]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=2000,  # 1000 мало, больше обычно бессмысленно: сходится на 2000
                           scale_pos_weight=1/df["Answer"].mean(),  # дисбаланс классов
                           learning_rate=0.01,
                           eval_metric='AUC', 
                           cat_features=["first_word", "second_word", "Question", "sentence"],
                           metric_period=1000,
                          )


model.fit(X_train,
          y_train,
          eval_set=(X_test, y_test),
          verbose=True)

0:	test: 0.7288020	best: 0.7288020 (0)	total: 261ms	remaining: 8m 40s
1000:	test: 0.8233418	best: 0.8233418 (1000)	total: 11.7s	remaining: 11.7s
1999:	test: 0.8256765	best: 0.8256765 (1999)	total: 24.2s	remaining: 0us

bestTest = 0.8256764904
bestIteration = 1999



In [135]:
X_train, X_test, y_train, y_test = train_test_split(df_with_emb[df_with_emb.columns[1:]], df_with_emb["Answer"], train_size=.8, random_state=1337)

model_emb = CatBoostClassifier(iterations=2000,
                           scale_pos_weight=1/df["Answer"].mean(),
                           learning_rate=0.01,
                           eval_metric='AUC', 
                           cat_features=["first_word", "second_word", "Question", "sentence"],
                           metric_period=1000,
                          )


model_emb.fit(X_train,
          y_train,
          eval_set=(X_test, y_test),
          verbose=True)

0:	test: 0.7072519	best: 0.7072519 (0)	total: 155ms	remaining: 5m 9s
1000:	test: 0.8231109	best: 0.8231109 (1000)	total: 19s	remaining: 18.9s
1999:	test: 0.8198939	best: 0.8231109 (1000)	total: 38.5s	remaining: 0us

bestTest = 0.8231109315
bestIteration = 1000

Shrink model to first 1001 iterations.


# Генерация ответов

In [124]:
test = pd.read_csv("data.csv", sep=";")
test.drop(df.index, inplace=True)
test.head()

,ID,Question
30000,30001,"Что в литературе обозначает слово ""эпитет""?"
30001,30002,Когда отмечается ДЕНЬ ШОКОЛАДА
30002,30003,Кто из этих бодибилдеров играл Халка
30003,30004,"Последняя на данный момент страна, официально ..."
30004,30005,Как зовут лютоволка Арьи Старк из сериала «Иг...


In [125]:
import string
import re
def remove_punct(text):
    table = str.maketrans({key: " " for key in string.punctuation})
    res = text.translate(table).lower()
    res = re.sub(r"\s+", " ", res)
    res = re.sub(r"\d+", "", res)
    return res

test["sentence"] = test["Question"].apply(remove_punct)

In [126]:
for name, func in tqdm(new_features.items()):
    test[name] = test["Question"].apply(func)

test.head()

  0%|          | 0/22 [00:00<?, ?it/s]

,ID,Question,sentence,text_len,fancy_quotes,regular_quotes,count_question_marks,count_excl_marks,count_dots,count_commas,...,first_word_len,second_word_len,uppercase_letters,has_bad_spaces,looks_bad,len_class_1,clever_vk,looks_good,ok_upper,ok_upper_text
30000,30001,"Что в литературе обозначает слово ""эпитет""?",что в литературе обозначает слово эпитет,9,0,1,1,0,0,0,...,3,1,1,0,1,0.136808,0,0,1,1
30001,30002,Когда отмечается ДЕНЬ ШОКОЛАДА,когда отмечается день шоколада,4,0,0,0,0,0,0,...,5,10,13,0,1,0.027194,0,0,0,0
30002,30003,Кто из этих бодибилдеров играл Халка,кто из этих бодибилдеров играл халка,6,0,0,0,0,0,0,...,3,2,2,0,1,0.090841,0,0,1,0
30003,30004,"Последняя на данный момент страна, официально ...",последняя на данный момент страна официально п...,11,0,0,0,0,0,1,...,9,2,2,0,1,0.135048,0,0,1,0
30004,30005,Как зовут лютоволка Арьи Старк из сериала «Иг...,как зовут лютоволка арьи старк из сериала «игр...,12,1,0,1,0,0,0,...,3,5,4,0,0,0.140865,0,1,1,1


In [127]:
test.columns

Index(['ID', 'Question', 'sentence', 'text_len', 'fancy_quotes',
       'regular_quotes', 'count_question_marks', 'count_excl_marks',
       'count_dots', 'count_commas', 'count_digits', 'ends_question_mark',
       'good_start_words', 'first_word', 'second_word', 'first_word_len',
       'second_word_len', 'uppercase_letters', 'has_bad_spaces', 'looks_bad',
       'len_class_1', 'clever_vk', 'looks_good', 'ok_upper', 'ok_upper_text'],
      dtype='object')

In [128]:
ans = model.predict_proba(test[test.columns[1:]])

In [129]:
idx = 30001
with open("submission_without_emb.csv", "w") as f:
    for proba_0, proba_1 in ans:
        print(f"{idx},{proba_1}", file=f)
        idx += 1

In [142]:
pd.DataFrame(q_emb_test_100)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-5.053438,-0.171971,-2.755329,2.369126,1.303877,0.181111,-2.215664,-0.168745,2.051039,1.247580,...,0.051443,0.289292,-0.108256,0.634285,0.079296,-0.024683,0.367339,0.062868,0.215675,-0.130157
1,-3.388886,4.019153,3.668693,0.577075,-0.039260,0.057817,-0.419019,1.434612,-0.028012,0.174652,...,0.966393,0.138811,-0.051004,-0.045422,-0.159528,-0.581222,-0.376053,-0.372870,-0.207992,0.261812
2,-3.309399,5.316131,2.423115,0.527937,-0.148207,-1.008696,2.520061,-1.712229,-0.556395,0.049202,...,0.780720,-0.376126,-0.310498,0.949654,-0.312943,-0.499143,0.724869,-0.370431,0.478213,0.097119
3,-2.499531,5.825404,0.770044,-1.840138,1.023180,1.493586,-0.023146,0.047245,-0.903025,0.960789,...,-0.604365,0.312750,-0.282736,-0.476776,-0.198463,0.147222,0.033906,0.078792,-0.907260,-1.145006
4,5.343839,-2.209629,1.440273,-0.487783,-0.387644,0.892042,0.134731,0.487677,1.046695,-0.573160,...,0.156518,-0.111652,-0.070506,-0.001688,0.077938,-0.042295,-0.038256,0.097594,0.058143,-0.093060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11082,7.737640,0.591610,-0.278839,0.489993,-0.357313,-0.167066,-0.510473,-0.239073,-0.262182,0.311051,...,-0.009307,0.253902,-0.032976,0.226959,-0.128190,0.063424,0.006270,-0.280610,0.018612,-0.279529
11083,-5.487819,-2.320698,2.542247,2.183891,-0.620578,0.439378,-1.241388,1.470586,0.155869,0.820708,...,-0.390704,-0.342750,-0.301761,0.178392,-0.167322,-0.083191,0.141908,0.146630,-0.526468,-0.212230
11084,-4.211458,-1.365688,-1.325038,0.216567,1.413504,0.921400,0.150034,1.132277,-0.218502,-1.844435,...,-0.260455,0.110908,-0.271326,-0.186117,0.142313,-0.467994,0.486052,0.002326,0.238862,0.480983
11085,-2.337895,5.339634,2.861571,-1.736470,-1.507650,2.671951,0.642664,3.372152,-0.447288,0.316430,...,-0.173279,-0.049195,-0.294425,0.277609,0.835248,0.474608,0.700101,0.388218,1.008087,1.069598


In [147]:
with open("q_emb_test.pickle", "rb") as f:
    q_emb_test = pickle.load(f)

q_emb_test = np.array([emb.detach().numpy() for emb in q_emb_test])
pca = PCA(n_components=100)
q_emb_test_100 = pca.fit_transform(q_emb_test)

test_with_emb = pd.concat([test.reset_index(drop=True), pd.DataFrame(q_emb_test_100)], axis=1)

In [148]:
test_with_emb

,ID,Question,sentence,text_len,fancy_quotes,regular_quotes,count_question_marks,count_excl_marks,count_dots,count_commas,...,90,91,92,93,94,95,96,97,98,99
0,30001,"Что в литературе обозначает слово ""эпитет""?",что в литературе обозначает слово эпитет,9,0,1,1,0,0,0,...,-0.293431,-0.139088,0.480196,0.044396,0.732216,-0.271315,0.603630,0.095044,-0.438201,-0.237259
1,30002,Когда отмечается ДЕНЬ ШОКОЛАДА,когда отмечается день шоколада,4,0,0,0,0,0,0,...,0.128100,-0.604454,-0.333918,0.346860,-0.458131,0.000746,-0.560871,0.541392,-0.632217,0.507058
2,30003,Кто из этих бодибилдеров играл Халка,кто из этих бодибилдеров играл халка,6,0,0,0,0,0,0,...,0.315314,0.044377,-0.054262,0.272611,1.239524,0.329442,0.372701,0.653212,-0.265747,0.298089
3,30004,"Последняя на данный момент страна, официально ...",последняя на данный момент страна официально п...,11,0,0,0,0,0,1,...,-0.322416,0.152302,-0.269907,-0.034038,-0.323438,0.162512,0.271435,0.384586,1.751705,-0.982121
4,30005,Как зовут лютоволка Арьи Старк из сериала «Иг...,как зовут лютоволка арьи старк из сериала «игр...,12,1,0,1,0,0,0,...,0.077749,0.161377,-0.093060,-0.004044,-0.017656,-0.022144,0.073079,0.075503,-0.155322,-0.085675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11082,41083,В каком году распался СССР,в каком году распался ссср,5,0,0,0,0,0,0,...,0.045519,-0.183759,0.402976,0.089826,0.198986,-0.146899,-0.154955,0.007242,0.048430,-0.223838
11083,41084,Сколько калорий в 100 гр арбуза?,сколько калорий в гр арбуза,7,0,0,1,0,0,0,...,-0.244581,0.011143,-0.192106,-0.094996,0.070583,-0.039099,0.075790,0.273077,0.096453,0.253249
11084,41085,"Сколько хвостов у лиса, который является демон...",сколько хвостов у лиса который является демоно...,12,0,0,1,0,0,1,...,-0.154643,-0.344913,-0.293285,0.024648,0.252123,0.172254,0.601745,0.310795,-0.287821,0.105476
11085,41086,Сколько раз магнитогорский металлург становилс...,сколько раз магнитогорский металлург становилс...,8,0,0,0,0,0,0,...,0.300420,0.635079,-0.295590,0.328709,1.141051,-0.046662,0.835193,-0.422337,-0.833262,1.063787


In [149]:
test_with_emb.columns

Index([                  'ID',             'Question',             'sentence',
                   'text_len',         'fancy_quotes',       'regular_quotes',
       'count_question_marks',     'count_excl_marks',           'count_dots',
               'count_commas',
       ...
                           90,                     91,                     92,
                           93,                     94,                     95,
                           96,                     97,                     98,
                           99],
      dtype='object', length=125)

In [150]:
ans_emb = model_emb.predict_proba(test_with_emb[test_with_emb.columns[1:]])

idx = 30001
with open("submission_emb.csv", "w") as f:
    for proba_0, proba_1 in ans:
        print(f"{idx},{proba_1}", file=f)
        idx += 1